In [1]:
from train import *
import matplotlib.pyplot as plt
import keras.backend as K
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from utils import dice_coef
from model.unet_v2 import Unet
from pathlib import Path

from keras.layers import MaxPool2D, Conv2D, Input, Dropout, Conv2DTranspose, concatenate
from keras import Model

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


## updated model

### constructor

Let's first check if we have our params right.

In [2]:
experiment_dir=Path('experiments/updated_unet')

In [3]:
params = Params(experiment_dir / 'params.json')

In [4]:
params.dict

{'learning_rate': 0.001,
 'steps_per_epoch': 100,
 'epochs': 40,
 'batch_size': 4,
 'start_epoch': 0,
 'last_step': 10,
 'batch_size_val': 1,
 'validation_steps': 3,
 'batch_size_test': 1,
 'input_shape': [512, 512, 1],
 'kernel_initializer': 'he_normal',
 'init_filters': 64,
 'dropout': 0.2}

In [5]:
unet = Unet(params)

In [6]:
unet.input_shape, unet.kernel_initializer, unet.init_filters, unet.dropout

([512, 512, 1], 'he_normal', 64, 0.2)

In [7]:
unet.config

{'kernel_size': (3, 3),
 'activation': 'relu',
 'padding': 'same',
 'pool_size': (2, 2),
 'kernel_size_transposed': (2, 2),
 'strides_transposed': (2, 2)}

### builder

Let's now check the builder.

In [8]:
input_layer = Input(shape=unet.input_shape, dtype='float32')

In [9]:
conv1 = unet.contracting_block(filters=unet.init_filters, 
                               in_layer=input_layer, is_max_pool=False)

In [10]:
params.init_filters = 32

In [11]:
unet = Unet(params)

In [12]:
model = unet.build_model()

In [13]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 512, 512, 32) 320         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 512, 512, 32) 9248        conv2d_3[0][0]                   
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 512, 512, 32) 0           conv2d_4[0][0]                   
____________________________________________________________________________________________